In [1]:
from hlathena import PeptideDatasetTrain

2023-12-18 15:44:30.702904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 15:44:32.996104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/app/python/3.7.4/lib:/n/app/openblas/0.2.19/lib:/n/app/gcc/6.2.0/lib64:/n/app/gcc/6.2.0/lib
2023-12-18 15:44:32.996283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/app/python/3.7.4/lib:/n/app/open

In [2]:
import pandas as pd

In [3]:
mini_df = pd.read_csv("../../notebooks/input/test_training.csv")

In [4]:
peptide_dataset = PeptideDatasetTrain(mini_df, allele_col_name='mhc', target_col_name='tgt', feat_cols=['tgt_weight'])


In [5]:
import torch.utils.data as torch_data
import torch
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import os
import datetime
import random

from hlathena import peptide_nn
from hlathena.training_evaluation import TrainingEvaluation

decoy_mul = 1
decoy_ratio = 1
# for fold in range(folds):
fold=0
feat_set = []
seed=1
folds=5
batch_size = 5000
epochs = 100
learning_rate = 0.01
pred_replicates = 10
dropout_rate = 0.1 #0.0
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [6]:
feature_dims = peptide_dataset.feature_dimensions()

peptide_dataset.reassign_folds(folds, seed=seed)


In [11]:
from torch import nn

# t = Timer()
# t.start()
eval_dict = {"peptide":[], "hla":[], "target":[], "pred_mean":[], "pred_var":[], "fold":[]}
            
print("Training fold {} of {}...".format(str(fold+1), folds))
resampling_hits=False
train_ids = peptide_dataset.get_train_idxs(fold=fold, decoy_mul=decoy_mul, resampling_hits=resampling_hits, seed=seed)
test_ids = peptide_dataset.get_test_idxs(fold=fold, decoy_ratio=decoy_ratio, seed=seed)
print(f"decoy mul: {decoy_mul}")
print(f"decoy ratio: {decoy_ratio}")


# Sample elements randomly from a given list of ids, no replacement.
train_subsampler = peptide_nn.PeptideRandomSampler(train_ids, seed) # TODO: can replace with SubsetRandomSampler and a torch generator
test_subsampler = peptide_nn.PeptideRandomSampler(test_ids, seed)
# return peptide_dataset, train_subsampler, test_subsampler, train_ids, test_ids
# Define data loaders for training and testing data in this fold
trainloader = torch_data.DataLoader(peptide_dataset, batch_size=batch_size, sampler=train_subsampler)
testloader = torch_data.DataLoader(peptide_dataset, batch_size=batch_size, sampler=test_subsampler)

try:
    val_subsampler = peptide_nn.PeptideRandomSampler([i for i in range(len(val_dataset))], seed)
    valloader = torch_data.DataLoader(val_dataset, batch_size=batch_size, shuffle=val_subsampler)
except:
    valloader = None
# t.stop("setup time")

# t.start()
model = peptide_nn.PeptideNN(feature_dims, dropout_rate)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.to(device)
optimizer_dict = peptide_nn.train(model, trainloader, learning_rate, epochs, device)#, valloader)
# t.stop("train time")

# # peptide_nn.save_model(model, fold, models_dir, configs_dir, optimizer_dict, model_config)

# t.start()
inputs, targets, indices, preds = peptide_nn.evaluate(model, testloader, pred_replicates, device)
inputs = torch.vstack(inputs).cpu()
targets = [t.item() for t in torch.hstack(targets).cpu()]
indices = [i.item() for i in torch.hstack(indices).cpu()]
preds = torch.vstack(preds).cpu()

# input_peps = [peptide_dataset.decode(p) for p in inputs]
input_peps, input_hla = [peptide_dataset.peptides[i] for i in indices], [peptide_dataset.hla[i] for i in indices]
pred_means = preds.mean(dim=-1).numpy()
pred_vars = preds.var(dim=-1).numpy()

for p, h, t, m, v in zip(input_peps, input_hla, targets, pred_means, pred_vars):
    eval_dict['peptide'].append(p)
    eval_dict['hla'].append(h)
    eval_dict['target'].append(t)
    eval_dict['pred_mean'].append(m)
    eval_dict['pred_var'].append(v)
    eval_dict['fold'].append(str(fold+1))

eval_df_nn2 = pd.DataFrame.from_dict(eval_dict)
# eval_df_nn2.to_csv('bigmhc/pephla_10allele_nn2_evaldf.csv',index=None)
# t.stop("eval time")


Training fold 1 of 5...
Resampling hits: False
decoy mul: 1
decoy ratio: 1


ValueError: not enough values to unpack (expected 4, got 3)